## 斷詞

目標:  
> 1. 將所有簡體字轉成繁體字
> 2. 斷詞
> 3. 計數各個詞的出現次數

參考自  
[[NLP][Python] 透過 ckiptagger 來使用繁體中文斷詞的最佳工具 CKIP](https://clay-atlas.com/blog/2019/09/24/python-chinese-tutorial-ckiptagger/)  
[[NLP][Python] 透過 opencc-python-reimplemented 套件來使用中文繁簡轉換工具 OpenCC](https://clay-atlas.com/blog/2019/09/24/python-chinese-tutorial-opencc/)

### 所有需要下載的套件

- ckiptagger 中研院的斷詞套件
- gdown 可以從 google drive 上載下模型

In [3]:
#! pip install opencc
#! pip install ckiptagger
#! pip install tensorflow
#! pip install gdown

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14775 sha256=cdd3eb949fae42698cfc90899a7888261b94002b8fcf13808c68670a7ecc269e
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\7d\37\b6\b2a79c75e898c0b8e46ff255102602d7159a10d9af0d80641a
Successfully built gdown
  Using cached tensorflow-2.8.0-cp39-cp39-win_amd64.whl (438.0 MB)
  Using cached protobuf-3.20.0-cp39-cp39-win_amd64.whl (904 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached tensorboard-2.8.0-py3-none-any.whl (5.8 MB)
  Using cached gast-0.5.3-py3-none-any.whl (19 kB)
  Using cached flatbuffers-2.0-py2.py3-none-any.whl (26 kB)
  Using cached o

ERROR: Could not install packages due to an OSError: [WinError 32] 程序無法存取檔案，因為檔案正由另一個程序使用。: 'C:\\Users\\User\\anaconda3\\Lib\\site-packages\\tensorflow\\python\\_pywrap_tensorflow_internal.pyd'
Consider using the `--user` option or check the permissions.



  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4847 sha256=17696a5692dad0e75e288f6b790e00914dec4de31a23b9241571cc4ac4e3a412
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\b6\0d\90\0d1bbd99855f99cb2f6c2e5ff96f8023fad8ec367695f7d72d
Successfully built termcolor


下載模型 data.zip 到我們當前的目錄底下, 每台電腦只要跑一次就好.  
如果下載有困難, 可以到[這裡](https://drive.google.com/drive/folders/105IKCb88evUyLKlLondvDBoh7Dy_I1tm)下載.  
**注意!!!** 檔案位置要跟這個 ipynb 檔在同一個位置.

In [4]:
#!from ckiptagger import data_utils
#!data_utils.download_data_gdown("./")

Downloading...
From: https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
To: C:\Users\User\Desktop\data.zip
100%|█████████████████████████████████████████████████████████████████████████████| 1.88G/1.88G [01:13<00:00, 25.4MB/s]


### 自行設定

In [2]:
#in_path = "C:/Users/User/Desktop/DATA2/rawdata_nocut/" # 檔案匯入位置
#out_path = "C:/Users/User/Desktop/cut/" # 檔案匯出位置

#in_path = "C:/Users/User/Desktop/DATA2/rawdata_nocut/" # 檔案匯入位置
#out_path = "C:/Users/User/Desktop/cut/" # 檔案匯出位置

in_path = "C:/Users/e2414/OneDrive/桌面/專題/AllData/rawdata_nocut/" # 檔案匯入位置
out_path = "C:/Users/e2414/OneDrive/桌面/專題/AllData/wordcut/finish_cut/rawnocut_cut/" # 檔案匯出位置

In [3]:
import warnings
warnings.filterwarnings("ignore")

### 匯入所需套件 & 檔案

In [1]:
from opencc import OpenCC
from ckiptagger import WS, POS, NER
import pandas as pd
import time

cc = OpenCC('s2tw')
ws = WS("data")
pos = POS("data")
ner = NER("data")

### 建立詞典 (一定要跑)

如果已知有特殊的詞, 可以先建立詞典, 會有比較正確的斷詞結果, 以下是範例, **需自行調整**.

In [6]:
from ckiptagger import construct_dictionary

# 下面是「建議」斷詞
word_to_weight = {
   #大福
   "韓國瑜": 2, "短總":3, "喜翻": 2, "好味小姐": 2, "口愛": 2,
    "本丸": 2, "腦粉": 2, "夢夢": 2, "奴才": 2, "水逆": 2,
    "聞氫哥": 2, "壹加壹": 2, "營養師": 2, "躺平": 2, "敲碗": 2, 
    "業配":3, "人設": 2, "惹毛": 2, "愛的迫降": 2, "歸剛欸": 2,
    "直播": 2, "祖克柏": 2, "型男": 2, "拍立得": 2, "撒嬌": 2,
    "米古": 2, "狸貓": 2, "韓總": 2, "饋桃" : 2, "噗攏共": 2,
    "佛地魔": 2, "起司": 2, "拿拿摳": 2, "拿拿扣": 2, "好味":3,
    "拿董": 2, "娘家": 2, "手軟": 2, "光一": 2, "靠北": 2,
    "倒霉": 2, "比特幣": 2, "確診": 2, "口以": 2, "元宇宙": 2,
    "玩命關頭": 2, "迷因": 2, "拿鐵": 2, "福袋": 2, "發爾麵": 2,
    "瀏海": 2,  "女鵝": 2, "狗勾": 2, "抹茶": 2, "科普": 2,
    "小必": 2, "訂餐": 2, "戴資穎": 2, "連假": 2, "抖內": 2,
    "炒作": 2, "小豬": 2, "頭號": 2, "辣個": 2, "比特犬": 2,
    "博客來": 2, "建達出奇蛋": 2, "節哀": 2, "拔罐": 2, "鯊小": 2,
    "茶碗蒸": 2, "持家男": 2, "塔塔": 2, "藍亦明": 2, "八三夭": 2,
    "隔離霜": 2, "康復": 2, "味醂": 2, "炎上": 2, "以牙還牙": 2,
    "實聯制": 2, "味春捲": 2, "古阿莫": 2, "狠愛演": 2, "奶昔": 2,
    "老蕭": 2, "鯛魚燒": 2, "雞蛋糕": 2, "出國": 2, "和服": 2, 
    "穿搭": 2, "千層": 2, "送禮自用兩相宜": 2, "蛋捲": 2, "抹茶": 2, 
    "喜年來": 2, "迷克夏": 2, "勞撫": 2, "打妹": 2, "回郵": 2, 
    "信封": 2, "襪靠": 2, "晚安": 2, "手收": 2, "搞剛": 2, 
    "燉肉": 2, "蔥抓餅": 2, "水煎包": 2, "牛小排": 2, "桌布": 2, 
    "下標": 2, "素顏": 2, "北比": 2, "巴黎萊雅": 2, "黑眼圈": 2, 
    "遮瑕筆": 2, "歐美妝": 2, "妝前乳": 2, "染眉膏": 2, "湯唯": 2, 
    "搞剛": 2, "完妝": 2, "妝感": 2, "定妝": 2, "真的": 2, 
    "遮瑕刷": 2, "棋盤格": 2, "粉底液": 2, "亮片": 2, "蘋果肌": 2, 
    "北鼻": 2, "歸剛": 2, "置入性": 2, "小朱": 2, "小李": 2, 
    "融化": 2, "即視感": 2, "茶葉蛋": 2, "孫藝珍": 2, "玫瑰瞳鈴眼": 2, 
    "行李": 2, "五月天": 2, "療癒": 2, "盜哥": 2, "傻眼": 2, 
    "載具": 2, "雞來速": 2, "曲家瑞": 2, "乳瑪琳": 2, "中國製": 2, 
    "奧客": 2, "植村秀": 2, "欸嚕": 2, "雨刷": 2, "雅詩蘭黛": 2, 
    "偶爸": 2, "把拔": 2, "不倒翁": 2, "楓之谷": 2, "充公": 2, 
    "過冬": 2, "微熱山丘": 2, "婊姐": 2, "順順利利": 2, "理膚寶水": 2, 
    "婊姊": 2, "小資": 2, "車輪餅": 2, "沙龍照": 2, "胖皮": 2, 
    "囤積控": 2, "綜口味": 2, "照騙": 2, "小賴": 2, "令傑": 2, 
    "貓砂": 2, "那英": 2, "畢旅": 2, "頭貼": 2, "南柱赫": 2, 
    "家純": 2, "擋泥板": 2, "表特版": 2, "平頭": 2, "生蛋": 2, 
    "產蛋": 2, "勤美": 2, "贖罪": 2, "吳奕凡": 2, "裴柱現": 2, 
    "中島美嘉": 2, "實至名歸": 2, "紀董": 2,
    
    # 柯南
    "產婦": 2, "甲意": 2, "考慮": 2, "一模模一樣樣": 2, "打線": 2,
    "悍將": 2, "有差": 2, "南無": 2, "阿彌陀佛": 2, "大願地藏王": 2,
    "問天": 2, "發揮": 2, "推薦": 2, "小梅雨": 2, "直澄": 2, 
    "宗介":3, "小芭內": 2, "我妻善逸": 2, "吐槽": 2, "伊黑小芭內": 2,
    "依麗莎白": 2, "我英": 2, "名柯": 2, "飆速宅男": 2, "琥珀": 2,
    "網王": 2, "火影": 2, "黑之契約者": 2, "哨聲響起" : 2, "中尾隆盛": 2,
    "迷你多啦": 2, "聲優": 2, "小紅多啦": 2, "了拉": 2, "推": 2,
    "廢天使加百列": 2, "柯南": 2, "紅與黑": 2, "大神": 2, "排球少年": 2,
    "兵長": 2, "神谷": 2, "肥嘟嘟佐衛門": 2, "不理不理左衛門": 2, "帝釋天": 2,
    "肥嘟嘟左衛門": 2, "臨也": 2, "間桐慎二": 2, "小遙": 2, "塔矢亮": 2,
    "生日快樂": 2,  "玉藻京介": 2, "鳴人": 2, "城之內": 2, "潮與虎": 2,
    "國王排名": 2, "中原茂": 2, "堀川亮": 2, "鍊金術師": 2, "北高": 2,
    "整天": 2, "屁眼": 2, "補血": 2, "將心比心": 2, "樓下": 2,
    "老司機": 2, "黑歷史": 2, "女鵝": 2, "把拔": 2, "翻牆": 2,
    "茶碗蒸": 2, "翹臀": 2, "療癒": 2, "虎哩": 2, "眼冒金星": 2,
    "平野惠一": 2, "黃燈初上": 2, "福來喜": 2, "同手同腳": 2, "跋山涉水": 2,
    "三修": 2, "雄女": 2, "好康": 2, "原潑": 2, "軌扶神宮": 2,
    "中二病": 2, "疊疊樂": 2, "爬竿": 2, "小編": 2, "一口": 2, 
    "瞬間": 2, "一探究竟": 2, "熊掌": 2, "差很多": 2, "全家": 2, 
    "阿莫伊漾": 2, "變髮圖強": 2, "差粉多": 2, "捅一": 2, "凶弟": 2, 
    "可怖": 2, "東融": 2, "不倒翁": 2, "出差": 2, "不期不待": 2, 
    "不得不說": 2, "蔥抓餅": 2, "桌曆": 2, "線上": 2, "若熙": 2, 
    "相知相惜": 2, "給讚": 2, "設計": 2, "干貝": 2, "業配": 2, 
    "一發": 2, "麻媽": 2, "清胃": 2, "冰奶": 2, "有情有義": 2, 
    "快打旋風": 2, "調適": 2, "芋頭控": 2, "猜心泡芙": 2, "抹茶":3, 
    "霸脫": 2, "許願": 2, "連假": 2, "好久": 2, "拜偷": 2, 
    "三條線": 2, "吃爆": 2, "花森": 2, "五迷": 2, "勾錐": 2, 
    "麵對": 2, "五月天": 2, "伍告水": 2, "伍影謀": 2, "嘖嘖": 2, 
    "奉還": 2, "餅總": 2, "愛老虎油": 2, "田中": 2, "接球": 2, 
    "單賣": 2, "元宇宙": 2, "安格拉治": 2, "同台": 2, "正解": 2, 
    "歐兜邁": 2, "學人精": 2, "甲斐": 2, "大坨": 2, "碰糖": 2, 
    "殺小": 2, "兩宿面癱": 2, "下船": 2, "不愁吃穿": 2, "咒迴": 2, 
    "五条": 2, "伏黑": 2, "五條": 2, "鬼滅": 2, "五條悟":3, 
    "敲碗": 2, "腳踏墊": 2, "全臺": 2, "全程": 2, "極簡": 2, 
    "大作戰": 2, "自然醒": 2, "玫瑰鹽": 2, "便便": 2, "會考": 2, 
    "冰導": 2, "腰酸背痛": 2, "各安其位": 2, "到府": 2, "控肉": 2,
    "手作": 2, "辣個": 2, "無菇": 2, "歐北": 2, "揪派": 2, 
    "銅鑼燒": 2, "體重計": 2, "共菇": 2, "東坡肉": 2, "封肉": 2, 
    "白吐司": 2, "哪次": 2, "末鹿狂花": 2, "無弓不狩鹿": 2, "浮誇": 2, "同類": 2, 
    "吸貓": 2, "佛系": 2, "手遊": 2, "森氣": 2, "嚕狗": 2, "樹懶": 2, "厚顏無恥": 2, "機歪": 2, "沒關需": 2,
    "章魚燒": 2, "吸睛": 2, "喝勝": 2, "小圓": 2, "濾鏡": 2,
    "浪流連": 2, "芬多精": 2, "美人胚": 2, "原廠": 2, "全聯": 2, 
    "數位":3, "熊大": 2, "散發": 2, "戴妝": 2, "華根初上": 2,
    "獅子王強大": 2, "小農": 2, "佛跳牆": 2, "立可白": 2, "俗女養成記": 2, "履歷": 2, "狗勾": 2, "人美心善": 2, "左鄰": 2,
    "右舍": 2, "左鄰右舍":3, "靠背": 2, "不共戴天": 2, "中二病": 2, 
    "大衛":3, "奇異博士": 2, "肉桂捲": 2, "無限期": 2, "蠟筆小新": 2,
    "時間線": 2, "海綿寶寶": 2, "成宮鳴": 2, "鑽石王牌": 2, "難波萬": 2,
    "無鉛": 2, "七大罪": 2, "善逸": 2, "雷之呼吸" : 2, "禰豆子": 2,
    "黃色小鴨": 2, "胡蝶忍": 2, "炭治郎": 2, "覆盆子": 2, "蘭姆酒": 2,
    "松露": 2, "腰內肉": 2, "三層肉": 2, "黑咖啡": 2, "粉多": 2,
    "跳跳虎": 2, "資料科學家": 2, "哈佛": 2, "大鼻": 2, "三花": 2,
    "魷魚遊戲": 2, "夾腳拖": 2, "芋泥": 2, "喜翻": 2, "肉鬆": 2,
    "店休": 2,  "回家": 2, "作育英才": 2, "機器學習": 2, "棒球大聯盟": 2,
    "大吾": 2, "吾郎": 2, "后宫團": 2, "一級棒": 2, "吾朗": 2,  "壽也": 2, "諸伏": 2, "乃木阪": 2, "黑衣组織": 2, "鬼滅之刃": 2,
    "不經": 2, "黑衣人": 2, "背水一戰": 2, "解封": 2, "每天": 2, "刀劍神域": 2, "涮乃葉": 2, "一番賞": 2, "喜番": 2,
    "無臉男": 2, "紅豬": 2, "天澤聖司": 2, "湯婆婆": 2, "千與千尋": 2, 
    "千尋":3, "掐玉": 2, "尋人啟事": 2, "刀與槍": 2, "尋寶": 2,
    "三小": 2, "北爛": 2, "欲擒故縱": 2, "極上": 2, "限動": 2,
    "首頁": 2, "全世界": 2, "豪好看": 2, "尋寶" : 2, "幸幸福福": 2, "上次": 2, "中二": 2, "妹控": 2, "賴床": 2, "千層": 2,
    "宅配": 2, "小山園": 2, "提拉米蘇": 2, "司康": 2, "貝果": 2,
    "宇治": 2, "滴妹": 2, "摸哥": 2, "獵魔士": 2, "素顏": 2,
    "王道": 2, "四季": 2, "囧大": 2, "摸媽": 2, "甲狀腺": 2, "鳴鳥不飛": 2,  "傲嬌": 2, "草你媽": 2, "握壽司": 2, "潮與虎": 2,
    "江老師": 2, "黑皮": 2, "信迪": 2, "咒術迴戰": 2, "尋人": 2, "真珠美人魚": 2, "售票": 2, "尋人啟示": 2, "百聽不厭": 2, "白眼": 2,
    "帥勾": 2, "說三道四": 2, "淚飆": 2, "金句良言": 2, "追蹤": 2, "軟件": 2, "北鼻": 2, "白眼": 2, "四季": 2, "上傳": 2, 
    "切八段": 2, "亦師亦友": 2, "伏地魔": 2, "鏈接": 2, "冏大": 2, 
    "摸摸": 2, "業配": 2, "烤肉": 2, "干貝": 2, "附餐": 2, "蝦搞": 2, "泡麵": 2, "涮嘴": 2, "豆乳雞": 2, "雞蛋糕": 2, 
    "和牛": 2, "九宮格": 2, "駁二": 2, "棧貳庫": 2, "外送": 2, "刀削麵": 2, "牛排": 2, "迷豆子": 2, "珊珊": 2, "乾": 2, 
    "菜比八": 2, "許甯築": 2, "沒事": 2, "直女": 2, "二遊": 2,"迷糊妹": 2, "新北": 2, "全國": 2, "力拼": 2, "免運": 2, 
    "地墊": 2, "小熊": 2, "烤肉夾": 2, "心心念念": 2, "心心念唸": 2, "首頁": 2, "霧都": 2, "提拉米蘇": 2, "寄居蟹": 2, "莫忘初衷": 2,
    "宇治": 2, "滴妹": 2, "摸哥": 2, "獵魔士": 2, "素顏": 2,"上英": 2,"南誠":2,"陳祥輝":2, "大福": 2,"大小姐": 2, "貝卡": 2,"歐易": 2,"貝爾": 2,"彭彭": 2,"品": 1,"柯南":2,
    "練團": 2, "脆樂團": 2, "累鼠": 2, "粗乃": 2, "圖文不符": 2, "生無可戀": 2, "春聯": 2, "彩排": 2, "念舊" : 2, "手搖": 2, "口愛": 2, "彩排": 2, "手搖杯": 3, "落山風": 2, "捷運": 2,	    
    "警察": 2, "返老還童": 2,

    # 貝爾
    "蝦味鮮": 2, "綠豆冰": 2, "楊枝甘露": 2, "即食麵": 2,"大福": 2,
    "一中": 2, "好市多": 2, "全家": 2, "喜來登": 2, "地獄谷": 2,
    "潘海利根": 2, "花都之水": 2, "小黑瓶": 2, "淡香精": 2, "極光水": 2,
    "水煮蛋": 2, "永生": 2, "活萃": 2, "平滑": 2,"扣扣": 2, "殼以餒": 2,
    "起乃": 2,"笑鼠": 2,"嚇屎": 2,"舒虎": 2,"綠茶婊": 2, "吃瓜": 2,
    "帶貨": 2,"歪果人": 2,"忙內": 2,"世一美": 2,"跑酷": 2,"母牛": 2,
    "藍眼淚": 2,"食蟻獸": 2,"祈錦鈅": 2,"哥吉拉": 2,"車銀優": 2,"東方神起": 2,
    "大眼仔": 2,"熊抱哥": 2,"三眼怪": 2,"毛怪": 2,"藍精靈": 2,"換換愛": 2,
    "惡魔在身邊": 2,"不良笑花": 2,"社交": 2,"李到硯": 2,"金在中": 2,"南柱赫": 2,
    "利特": 2,"機智醫生生活": 2,"步步惊心丽": 2,"五月的青春": 2,"簽名照": 2,
    "可旬": 2,"歹戲拖棚": 2,"三小": 2,"恰北北": 2,"蝦毀": 2,"靠腰": 2,
    "休旦幾累": 2,"群聚": 2,"赦免": 2,"物以類聚": 2,"裝瞎": 2,"似曾相似": 2,
    "花俏": 2,"以和為貴": 2,"管東": 2,"管西": 2,"登對": 2,"三觀": 2,"惡魔": 2,
    "博君一笑": 2,"碰瓷": 2,"軟爛": 2,"以偏概全": 2,"文創": 2,"不見": 2,"不散": 2,
    "拜天拜地": 2,"行禮": 2,"惹的禍": 2,"另謀": 2,"安康": 2,"完整版": 2,"趴著": 2,
    "失禮": 2,"折扣碼": 2,"詠春": 2,"團購": 2,"年假": 2,"大吉": 2,"桌遊": 2,
    "手游": 2,"換位": 2,"思考": 2,"一試成主顧": 2,"把走": 2,"泰奶": 2,"一絲": 2,
    "小編": 2,"抄襲": 2,"吃好吃滿": 2,"情深深": 2,"好可愛": 2,"好漂亮": 2,
    "好美": 2,"漂釀": 2,"全館": 2, "大好禮": 2,"提袋": 2,"美容油": 2,"包吃": 2,
    "丸子頭": 2, "湯圓頭": 2, "大逛特逛": 2,"北極熊": 2,"毛小孩": 2,"友善": 2,
    "極地": 2, "極光": 2, "玩爆": 2, "行大運": 2,"亂打": 2,"一桿進洞": 2,
    "半獸人": 2, '起崩':2, '幾霸昏':2,
    '華燈':2, '華燈初上':2, '夯劇':2, '奇亞籽':2, '絕絕子':2,
    '破防':2, '雙滅':2, '太絕了':2, '吳慷仁':2, '郭雪芙':2,
    '耳圈染':2, '淚牛滿面':2, '阿北':2, '美有姬':2, '撒拉嘿':2,
    '姨母笑':2, '繭居族':2, '咩噗':2, '麥客氣':2, '快樂星球':2,
    '天菜':2, '干飯人':2, '抖音':2, '吃播主':2, '吃播':2, '社畜':2,
    '萊納':2, '八哩八告':2, '史密斯':2, '甘阿捏':2, '真香':2,
    '芒果乾':2, '跑男':2, '興奮到模糊':2, '三保':2, '狐獴':2,
    '金銀淑':2, '具俊曄':2, '厄瓜多爾':2, '咕嚕咕嚕':2, '沒路用':2,
    '汁妹':2, '虧妹':2, '洗洗睏':2, '粗乃丸':2, '錘子':2, '買可樂':2,
    '喔某':2, '笑芬':2, '甘阿捏':2, '甘阿餒':2, '佛系':2, '北七':2,
    '史密斯':2, '邊緣人':2, '阿災':2, '災辣':2, '領便當':2, '魯蛇':2,
    '海景第一排':2, '住海邊':2, '撿到槍':2, '中央空調':2, '牙起來':2,
    '可撥':2, '咖啡話':2, '嘿啊':2, '雨都':2, '天龍國':2, '天龍人':2,
    '安安':2, '防疫旅館':2, '尬聊':2, '小姐姐':2, '小哥哥':2, '北車':2,
    '高捷':2, '北捷':2 , '中捷':2, '隱眼':2, '大奶微微':2, '熱那':2,
    '永豆':2, '小廢包':2, '水逆':2, '五木':2, '做好坐滿':2, '暴雷':2,
    '鞋感':2, '尬電':2, '高爾宣':2, '網美':2, '網紅':2, '龜山島':2,
    '斑比山丘':2, '十鼓仁糖':2, '清境':2, '張阿美農場':2, '每次':2,
    '睡不著':2, '整場':2, '達人':2, '大玩特玩':2, '牢靠':2, '活著':2,
    '大馬士革玫瑰':2, '韓式烤肉':2, '章魚燒':2, '團購價':2, '代餐':2,
    '蕁麻疹':2, '尋麻疹':2, '包子臉':2, '無薪假':2, '冰炫風':2, 
    '班上':2, '真好':2, '存在感':2, '全系列':2, '高跟鞋':2, '低跟':2,
    '每天':2, '做莫':2, '虎虎生威':2, '一山不能容二虎':2, '行大運':2,
    '對戒':2, '全站':2, '全天':2, '全天':2, '全體':2, '露背':2, 
    '從頭笑到尾':2, '路易莎':2, '機概':2, '數統':2, '市調':2, 
    '抽調':2, '電概':2, '線代':2, '企管':2, '地政':2, '社學':2,
    '大統盃':2, '對墻':2, '嗆死':2, '接噴':2, '快攻':2, '漢堡肉':2,
    '維尼熊':2, '跳跳虎':2, '伊爾':2, '全臺':2, '輕旅行':2, '絕美':2,
    '絕色':2, '租車':2, '抽好禮':2, '隨堂考':2, '隨堂測驗':2, '足浴包':2,
    '足浴':2, '一夜好眠':2, '湖畔':2, '減糖':2, '一日遊':2,
    '重溫':2, '徹夜不眠':2, '不含':2, '不安':2, '不錯':2, '下一站幸福':2,
    '意難忘':2, '臺啤':2, '奮起湖':2, '大冪冪':2, '三生三世十里桃花':2,
    '毛小孩':2, '文青':2, '燈會':2, '口舌之欲':2, '好逼人':2,
    '網拍':2, '必拍':2, '花中之王':2, '歐舒丹':2, '舒芙蕾':2,
    '冬陰海鮮湯':2, '冬陰':2, '巨好吃':2, '爆棚':2, '鎖死':2,
    '髮根燙':2, '挑染':2, '京喚羽修護系統':2, '自然捲':2, '角蛋白':2,
    '內層染':2, '一刀剪':2, '歐尼':2, '蜜桃唇':2, '香奈兒':2, '又純又慾':2,
    '雅詩蘭黛':2, '夢寐以求':2, '沙巴仙本那':2, '西雅圖':2, '知更鳥':2,
    '派對':2, 
    #粵語
    "呢個": 2,"偷食": 2,"諗起": 2,
    
    #歐易
    "韓國瑜": 2, "腦粉": 2, "夢夢": 2,  "奴才": 2, "水逆": 2,
    "聞氫哥": 2, "壹加壹": 2, "營養師": 2, "躺平": 2, "敲碗": 2, 
    "業配":  3, "人設": 2, "惹毛": 2, "愛的迫降": 2, "歸剛欸": 2,
    "直播": 2, "祖克柏": 2, "型男": 2, "拍立得": 2, "撒嬌": 2,
    "米古": 2, "狸貓": 2, "韓總": 2, "饋桃" : 2, "噗攏共": 2,
    "佛地魔": 2, "起司": 2, "娘家": 2, "手軟": 2, "光一": 2, 
    "靠北": 2, "倒霉": 2, "確診": 2, "口以": 2, "元宇宙": 2,
    "玩命關頭": 2, "迷因": 2, "拿鐵": 2, "福袋": 2, "發爾麵": 2,
    "瀏海": 2,  "女鵝": 2, "狗勾": 2, "抹茶": 2, "科普": 2,
    "小必": 2, "訂餐": 2, "戴資穎": 2, "連假": 2, "抖內": 2,
    "炒作": 2, "小豬": 2, "頭號": 2, "辣個": 2, "比特犬": 2,
    "拔罐": 2, "鯊小": 2, "三小": 2, "工三小": 2, "工沙小": 2,
    "茶碗蒸": 2, "持家男": 2, "塔塔": 2, "藍亦明": 2, "八三夭": 2,
    "隔離霜": 2, "康復": 2, "味醂": 2, "炎上": 2, "以牙還牙": 2,
    "實聯制": 2, "老蕭": 2, "苡任":2, "矮妹": 2, "大小姐": 2, "鯨華": 2,
    "小牛": 2,"邰哥": 2, "邰桑": 2, "阿啾": 2, "小黑": 2, "凱莉": 2,
    "柯南": 2, "貝爾": 2, "木曜": 2, "上不": 2, "蔡波能": 2, "蔡桃貴": 2,
    "鹿人": 2, "小慢": 2, "阿啾": 2, "奎丁": 2, "上班不要看": 2, "木曜": 2,
    "木曜四": 2, "邰照明": 2, "坤達": 2, "溫溫": 2, "溫泉蛋": 2, "泱泱": 2,
    "泱苗": 2, "阿部":2, "阿部瑪利亞":2, "溫妮": 2, "阿憲": 2, "阿民": 2,
    "安安邊緣子": 2, "麻姑師太": 2, "老周": 2, "野人": 2, "東區妹仔": 2, 
    "百靈果": 2, "眼球中央電視台": 2, "林書荷": 2, "外援": 2
}
# 下面是「強制」斷詞
word_to_weight2 = {
    "業配": 8, ",": 30,"丶": 30
}
dictionary = construct_dictionary(word_to_weight)
dictionary2 = construct_dictionary(word_to_weight2)

### 簡體轉繁體 + 斷詞 + 計數

* 斷詞結果將放在第 4 行, Cut 這裡.
* 將匯出 c + ID.xlsx --> 斷詞結果
* 將匯出 i + ID.xlsx --> 計數結果


### 如果有人覺得他一直跳訊息很煩, 下面的程式可以讓它不要跳

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [14]:
time_start = time.time() # 開始計時
start_ID = 200    # 檔案起始 ID
end_ID = 299 # 檔案結束 ID

for i in range(start_ID, (end_ID + 1)):
          
    # 匯入檔案
    file = "raw_nocut_" + str(i) + ".xlsx"
    df = pd.read_excel(in_path + file)
    df['Cut'] = 0
    df['POS'] = 0
    df['NER'] = 0
    index = []

    # 簡轉繁 & 斷詞
    for s in range(df.shape[0]):
        df['Cut'][s] = cc.convert(str(df['content'][s]))
        df['Cut'][s] = ws([df['Cut'][s]], recommend_dictionary = dictionary, coerce_dictionary = dictionary2)[0]
        df['POS'][s] = pos([df['Cut'][s]])
        df['NER'][s] = ner([df['Cut'][s]], df['POS'][s])
        
        index = index + df['Cut'][s]
        if (s == df.shape[0]-1):
            time_end = time.time() 
            timespend = (time_end- time_start)/60
            print('檔案 ', i, " 已處理完成. ", "時間: ", round(timespend, 2), " 分鐘")
    
    df = df.drop("Unnamed: 0", axis = 1)

    # 計數
    index = pd.DataFrame(index)
    index = index[0].value_counts()
    index = pd.DataFrame(index)
    index.columns = [i]
    index.insert(0, 'Word', index.index)
    
    # 匯出檔案
    df.to_excel(out_path + f"rawno_c{i}.xlsx", index= False)
    index.to_excel(out_path + f"rawno_i{i}.xlsx", index= False)
    
    
#time_start = time.time() # 開始計時
start_ID = 800 # 檔案起始 ID
end_ID = 811 # 檔案結束 ID


for i in range(start_ID, (end_ID + 1)):
          
    # 匯入檔案
    file = "raw_nocut_" + str(i) + ".xlsx"
    df = pd.read_excel(in_path + file)
    df['Cut'] = 0
    df['POS'] = 0
    df['NER'] = 0
    index = []

    # 簡轉繁 & 斷詞
    for s in range(df.shape[0]):
        df['Cut'][s] = cc.convert(str(df['content'][s]))
        df['Cut'][s] = ws([df['Cut'][s]], recommend_dictionary = dictionary, coerce_dictionary = dictionary2)[0]
        df['POS'][s] = pos([df['Cut'][s]])
        df['NER'][s] = ner([df['Cut'][s]], df['POS'][s])
        
        index = index + df['Cut'][s]
        if (s == df.shape[0]-1):
            time_end = time.time() 
            timespend = (time_end- time_start)/60
            print('檔案 ', i, " 已處理完成. ", "時間: ", round(timespend, 2), " 分鐘")
    
    df = df.drop("Unnamed: 0", axis = 1)

    # 計數
    index = pd.DataFrame(index)
    index = index[0].value_counts()
    index = pd.DataFrame(index)
    index.columns = [i]
    index.insert(0, 'Word', index.index)
    
    # 匯出檔案
    df.to_excel(out_path + f"rawno_c{i}.xlsx", index= False)
    index.to_excel(out_path + f"rawno_i{i}.xlsx", index= False)
    
    

start_ID = 1200 # 檔案起始 ID
end_ID = 1202 # 檔案結束 ID

for i in range(start_ID, (end_ID + 1)):
          
    # 匯入檔案
    file = "raw_nocut_" + str(i) + ".xlsx"
    df = pd.read_excel(in_path + file)
    df['Cut'] = 0
    df['POS'] = 0
    df['NER'] = 0
    index = []

    # 簡轉繁 & 斷詞
    for s in range(df.shape[0]):
        df['Cut'][s] = cc.convert(str(df['content'][s]))
        df['Cut'][s] = ws([df['Cut'][s]], recommend_dictionary = dictionary, coerce_dictionary = dictionary2)[0]
        df['POS'][s] = pos([df['Cut'][s]])
        df['NER'][s] = ner([df['Cut'][s]], df['POS'][s])
        
        index = index + df['Cut'][s]
        if (s == df.shape[0]-1):
            time_end = time.time() 
            timespend = (time_end- time_start)/60
            print('檔案 ', i, " 已處理完成. ", "時間: ", round(timespend, 2), " 分鐘")
    
    df = df.drop("Unnamed: 0", axis = 1)

    # 計數
    index = pd.DataFrame(index)
    index = index[0].value_counts()
    index = pd.DataFrame(index)
    index.columns = [i]
    index.insert(0, 'Word', index.index)
    
    # 匯出檔案
    df.to_excel(out_path + f"rawno_c{i}.xlsx", index= False)
    index.to_excel(out_path + f"rawno_i{i}.xlsx", index= False)
    
    
    
start_ID = 1204 # 檔案起始 ID
end_ID = 1265 # 檔案結束 ID

for i in range(start_ID, (end_ID + 1)):
          
    # 匯入檔案
    file = "raw_nocut_" + str(i) + ".xlsx"
    df = pd.read_excel(in_path + file)
    df['Cut'] = 0
    df['POS'] = 0
    df['NER'] = 0
    index = []

    # 簡轉繁 & 斷詞
    for s in range(df.shape[0]):
        df['Cut'][s] = cc.convert(str(df['content'][s]))
        df['Cut'][s] = ws([df['Cut'][s]], recommend_dictionary = dictionary, coerce_dictionary = dictionary2)[0]
        df['POS'][s] = pos([df['Cut'][s]])
        df['NER'][s] = ner([df['Cut'][s]], df['POS'][s])
        
        index = index + df['Cut'][s]
        if (s == df.shape[0]-1):
            time_end = time.time() 
            timespend = (time_end- time_start)/60
            print('檔案 ', i, " 已處理完成. ", "時間: ", round(timespend, 2), " 分鐘")
    
    df = df.drop("Unnamed: 0", axis = 1)

    # 計數
    index = pd.DataFrame(index)
    index = index[0].value_counts()
    index = pd.DataFrame(index)
    index.columns = [i]
    index.insert(0, 'Word', index.index)
    
    # 匯出檔案
    df.to_excel(out_path + f"rawno_c{i}.xlsx", index= False)
    index.to_excel(out_path + f"rawno_i{i}.xlsx", index= False)

檔案  204  已處理完成.  時間:  1.98  分鐘
檔案  205  已處理完成.  時間:  5.72  分鐘
檔案  206  已處理完成.  時間:  6.75  分鐘
檔案  207  已處理完成.  時間:  8.48  分鐘
檔案  208  已處理完成.  時間:  9.61  分鐘
檔案  209  已處理完成.  時間:  11.15  分鐘
檔案  210  已處理完成.  時間:  14.19  分鐘
檔案  211  已處理完成.  時間:  15.91  分鐘
檔案  212  已處理完成.  時間:  17.37  分鐘
檔案  213  已處理完成.  時間:  19.26  分鐘
檔案  214  已處理完成.  時間:  20.86  分鐘
檔案  215  已處理完成.  時間:  21.77  分鐘
檔案  216  已處理完成.  時間:  23.14  分鐘
檔案  217  已處理完成.  時間:  23.66  分鐘
檔案  218  已處理完成.  時間:  25.99  分鐘
檔案  219  已處理完成.  時間:  28.16  分鐘
檔案  220  已處理完成.  時間:  31.27  分鐘
檔案  221  已處理完成.  時間:  34.48  分鐘
檔案  222  已處理完成.  時間:  38.71  分鐘
檔案  223  已處理完成.  時間:  39.53  分鐘
檔案  224  已處理完成.  時間:  40.91  分鐘
檔案  225  已處理完成.  時間:  42.21  分鐘
檔案  226  已處理完成.  時間:  43.42  分鐘
檔案  227  已處理完成.  時間:  44.6  分鐘
檔案  228  已處理完成.  時間:  45.41  分鐘
檔案  229  已處理完成.  時間:  46.37  分鐘
檔案  230  已處理完成.  時間:  47.26  分鐘
檔案  231  已處理完成.  時間:  48.08  分鐘
檔案  232  已處理完成.  時間:  51.63  分鐘
檔案  233  已處理完成.  時間:  53.03  分鐘
檔案  234  已處理完成.  時間:  58.39  分鐘
檔案  235  已處理完成

In [16]:
in_path = "C:/Users/User/Desktop/DATA2/" # 檔案匯入位置
out_path = "C:/Users/User/Desktop/cut/" # 檔案匯出位置

start_ID = 812 # 檔案起始 ID
end_ID = 818 # 檔案結束 ID

for i in range(start_ID, (end_ID + 1)):
          
    # 匯入檔案
    file = str(i) + ".xlsx"
    df = pd.read_excel(in_path + file)
    df['Cut'] = 0
    df['POS'] = 0
    df['NER'] = 0
    index = []

    # 簡轉繁 & 斷詞
    for s in range(df.shape[0]):
        df['Cut'][s] = cc.convert(str(df['content'][s]))
        df['Cut'][s] = ws([df['Cut'][s]], recommend_dictionary = dictionary, coerce_dictionary = dictionary2)[0]
        df['POS'][s] = pos([df['Cut'][s]])
        df['NER'][s] = ner([df['Cut'][s]], df['POS'][s])
        
        index = index + df['Cut'][s]
        if (s == df.shape[0]-1):
            time_end = time.time() 
            timespend = (time_end- time_start)/60
            print('檔案 ', i, " 已處理完成. ", "時間: ", round(timespend, 2), " 分鐘")
    
    #df = df.drop("Unnamed: 0", axis = 1)

    # 計數
    index = pd.DataFrame(index)
    index = index[0].value_counts()
    index = pd.DataFrame(index)
    index.columns = [i]
    index.insert(0, 'Word', index.index)
    
    # 匯出檔案
    df.to_excel(out_path + f"c{i}.xlsx", index= False)
    index.to_excel(out_path + f"i{i}.xlsx", index= False)

檔案  807  已處理完成.  時間:  429.15  分鐘
檔案  808  已處理完成.  時間:  429.56  分鐘
檔案  809  已處理完成.  時間:  430.38  分鐘
檔案  810  已處理完成.  時間:  431.85  分鐘
檔案  811  已處理完成.  時間:  434.79  分鐘
